# レポート課題：量子系の時間発展のシミュレーションから、エネルギーを推定する

一般的に量子系のエネルギー固有値（例えば、最小エネルギー固有値=基底エネルギー）を計算することは難しい。しかし、プログラマブルな量子コンピュータがあれば、量子系のエネルギーを推定することができます。ここでは、量子系のエネルギー時間発展をシミュレーションすることで、エネルギーを推定してみたいと思います。

## 課題1:数学的な準備
### 課題1-1
初期状態としてある量子状態$|\psi _{\rm ini}\rangle$が与えられているとする。この状態に対して、ハミルトニアン、$H$での時間発展を用いて、ハミルトニアン$H$の最小エネルギー固有値を推定したいとしよう。この時，ハミルトニアン$H$の固有状態と固有エネルギーをそれぞれ$|E_i \rangle$と$E_i$とする．初期状態をエネルギー固有状態で展開すると、
$$
|\psi _{\rm ini} \rangle = \sum _i c_i |E_i \rangle 
$$
とすると、時刻$t$での状態$|\psi (t)\rangle = e^{-iHt} |\psi _{\rm ini} \rangle $はエネルギー固有値$E_i$を用いてどのように表されるか．（latexを用いた数式でこここに追記する形で提出してください）

### 課題1-2
この状態と初期状態との内積
$$
\langle \psi _{\rm ini} | \psi (t) \rangle 
$$
を計算するとどのような形になるか．（latexを用いた数式でこここに追記する形で提出してください）

### 考察（これは課題ではありません．上記の課題を問いあた後に得られる式の解説です）
このことから，初期状態と時刻$t$の状態の内積の値は，$-E_i$の周波数を持った波が重ね合わさった時系列データになっていることがわかる．これを利用すれば，時系列データのフーリエ変換を計算することで，どのような周波数成分が含まれているかを推定しエネルギーを推定することができる．

### 課題1-3
フーリエ変換を用いて周波数を推定するためには，時系列データに求めたい周波数の成分を十分大きく含まれている必要がある．これは，初期状態がどのような状態である必要があるか，展開係数$c_i$に言及して説明せよ．


## 課題２:エネルギー固有値をとめるハミルトニアンの定義とnumpyを使った対角化

ランダムにハミルトニアンを生成し、`qulacs observable` として定義します。

In [127]:
import numpy as np
import matplotlib.pyplot as plt
from qulacs import QuantumState, Observable, QuantumCircuit
from qulacs.state import inner_product, tensor_product

# 4体のパウリ演算子までを含むハミルトニアン
# n_qubits 量子ビット数
# ハミルトニアンに含まれる項の数
def generate_random_hamiltonian(n_qubits, num_terms):
    pauli_terms = ['I', 'X', 'Y', 'Z']
    hamiltonian = Observable(n_qubits)

    for _ in range(num_terms):
        num_paulis = np.random.randint(1, 5)  # Number of Pauli operators (up to 4)
        coefficient = np.random.uniform(-1, 1)  # Random coefficient between -1 and 1
        term = ""

        for i in range(n_qubits):
            if num_paulis > 0 and np.random.rand() > 0.5:
                pauli_op = np.random.choice(pauli_terms[1:])  # Random Pauli (X, Y, Z)
                term += f"{pauli_op} {i} "
                num_paulis -= 1

        if term.strip():  # Check if term is not empty
            hamiltonian.add_operator(coefficient, term.strip())

    return hamiltonian

In [128]:
nqubits = 2  # 量子ビット数
num_terms = 10 #項の数
hamiltonian = generate_random_hamiltonian(nqubits, num_terms)
print(hamiltonian)

0.407333 Y 0 + -0.534041 X 1 + -0.969555 Z 0 + -0.196327 Y 1 + 0.653626 Y 1


答え合わせができるように、`numpy array`に一度変換し、numpyの対角化を用いてエネルギー固有値を全て事前に計算しておきます。

In [129]:
# qulacs observable を numpy arrayに変換する関数
def observable_to_numpy_array(observable, n_qubits):
    matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=complex)
    for i in range(observable.get_term_count()):
        coef = observable.get_term(i).get_coef()
        pauli_id_list = observable.get_term(i).get_pauli_id_list()
        pauli_index_list = observable.get_term(i).get_index_list()
        temp_operator = Observable(n_qubits)
        temp_operator.add_operator(coef, " ".join(f"{['I', 'X', 'Y', 'Z'][pauli_id]} {index}" for pauli_id, index in zip(pauli_id_list, pauli_index_list)))
        matrix += temp_operator.get_matrix()
    return matrix

In [132]:
hamiltonian_array = observable_to_numpy_array(hamiltonian,nqubits)

### 課題2-1 
変換された numpy arrayの行列を `np.linalg.eigh()`を用いて固有値と固有ベクトルを計算し，固有値を全て書き出し，その最小値を求めよ．ただし、第一成分は固有値の配列、第二成分は固有ベクトルの配列となっている．
`eigenvalues, eigenvectors = np.linalg.eigh(対角化したい行列をここに入れる)`

### 後処理
エネルギーが負の値を取ると議論が複雑になるので、今回は適当にエネルギーをシフトしておき、エネルギー固有値が全て正になるようにしておく．

In [134]:
energy_shift = 6
#ハミルトニアンのエネルギーのシフト
hamiltonian.add_operator(energy_shift ,f"I {0}")      

### 課題2-2
シフトされたハミルトニアンのエネルギー固有値を2-1と同様に計算し、エネルギーが正になっていることを確認せよ．このシフトされたエネルギーの最小値が今求めたい値の答えである．

## 課題3:最小エネルギー固有状態の高い確率で含む初期状態の生成

最小エネルギー状態（基底状態）の固有値を推定したければ，初期状態が対応する固有状態の成分を十分に含んでいる必要がある．このような状態を生成する方法として，第五回の授業で利用したQAOAを用いよう．横磁場項と基底状態を知りたい問題ハミルトニアンを用いて、変分法によってエネルギー期待値が最小化されるような量子回路を作る．

### 課題3-1:以下を実行せよ．

In [ ]:
import scipy.optimize
from qulacs import QuantumState
from qulacs.gate import H,S,Sdag, sqrtX,sqrtXdag,sqrtY,sqrtYdag #1量子ビット Clifford演算
from qulacs import QuantumCircuit
from qulacs.state import inner_product, tensor_product

#横磁場項の追加
def add_X_fields(operator):
    nqubits = operator.get_qubit_count()
    for k in range(nqubits):
        operator.add_operator(1.0,"X {0}".format(k)) 
    return operator

#ハミルトニアンの期待値（イジング模型のエネルギーの期待値）を計算
def cost_QAOA(parameter_list,hamiltonian,driver_hamiltonian,num_layers):
    nqubits = hamiltonian.get_qubit_count()
    state = QuantumState(nqubits)
    
    for i in range(nqubits):
        H(i).update_quantum_state(state)
    define_QAOA_ansatz(parameter_list,hamiltonian,driver_hamiltonian,num_layers).update_quantum_state(state)
    return hamiltonian.get_expectation_value(state)

def define_QAOA_ansatz(parameter_list,Ising_hamiltonian,driver_hamiltonian,num_layers):
        nqubits = Ising_hamiltonian.get_qubit_count()
        circuit=QuantumCircuit(nqubits)
        for i in range(num_layers):
            circuit.add_observable_rotation_gate(Ising_hamiltonian,parameter_list[2*i],1)
            circuit.add_observable_rotation_gate(driver_hamiltonian,parameter_list[2*i+1],1)
        return circuit    

#レイヤー数
layer = 10

#driverハミルトニアンの定義
driver_hamiltonian = Observable(nqubits)
driver_hamiltonian = add_X_fields(driver_hamiltonian)

def cost(parameter_list):
    return cost_QAOA(parameter_list,hamiltonian,driver_hamiltonian,layer)

    
cost_history = []

# 黒魔術で初期パラメータを決めておく（かなり良い初期値からスタートできる）
alpha_Ising = 0.3
alpha_z = 1.2

bias_Ising= 0.2
bias_z = 1.5

gamma = [-bias_Ising- (alpha_Ising/layer)*i for i in range(layer)]
beta = [bias_z - (alpha_z/layer) * i for i in range(layer)]
init_theta_list = []
for i in range(layer):
    init_theta_list.append(gamma[i])
    init_theta_list.append(beta[i])
    
    

#init_theta_list = [random.random() for i in range(layer*2)]


cost_history.append(cost(init_theta_list))
method = "BFGS"
options = {"disp": True, "maxiter": 200, "gtol": 1e-3}


opt = scipy.optimize.minimize(cost, init_theta_list,
               method=method,
               callback=lambda x: cost_history.append(cost(x)),options = options)

plt.rcParams["font.size"] = 18
plt.plot(cost_history, color="red", label="QAOA")
plt.xlabel("Iteration")
plt.ylabel("cost function expectation value")
plt.legend()
plt.show()


実行後、最終的に見つかった回路のパラメータは、`opt.x`に格納されている．このパラメータを用いて定義される量子回路は
`define_QAOA_ansatz(opt.x,hamiltonian,driver_hamiltonian,layer)`なのでこの回路を用いて`optimized_state`に基底状態を多く含むであろう状態を生成しておく。

In [140]:
optimized_state = QuantumState(nqubits)

for i in range(nqubits):
        H(i).update_quantum_state(optimized_state)

define_QAOA_ansatz(opt.x,hamiltonian,driver_hamiltonian,layer).update_quantum_state(optimized_state)

## 課題4:時系列データのフーリエ変換によってエネルギー最小値を推定する

### 課題4-1：先ほど準備した状態を初期状態として、時間発展演算子を作用させ、初期状態との内積を計算し、時系列データ（実部と虚部）をプロットするグラフを表示せよ．

In [ ]:
#時間発展を作用させる状態
state = QuantumState(nqubits)

#初期状態を先ほど計算した状態に設定 |psi _ini>
ini_state = QuantumState(nqubits)
ini_state = optimized_state.copy()

#時間発展を作用させる状態を初期化 |psi(0)> = |psi _ini>
state=ini_state.copy()

# 時間発展のパラメータ
T = 100.0         # 全時間
N_t = 1000       # 時間ステップ数
dt = T / N_t     # 時間ステップ幅
num_trotter_steps = 100 #トロッター分割数
times = np.linspace(0, T, N_t)


# 時間発展ゲートの作成
time_evolution = QuantumCircuit(nqubits)
time_evolution.add_observable_rotation_gate(hamiltonian,-2*dt,num_trotter_steps)

# エネルギーの時系列データを保存する配列
overlap_re = np.zeros(N_t)
overlap_im = np.zeros(N_t)

# 時間発展とエネルギー計算のループ
for i in range(N_t):
    # エネルギーの期待値を計算
    overlap_re[i] = inner_product(ini_state,state).real
    overlap_im[i] = -inner_product(ini_state,state).imag #エネルギーが正の周波数になるように、複素共役をとっておく
    # 状態の時間発展
    time_evolution.update_quantum_state(state)

plt.plot(times,overlap_re)
plt.plot(times,overlap_im)
plt.show()


### 課題4-2 
以下のフーリエ変換を実行し，基底状態のエネルギー（最も小さな周波数成分）を推定し，対角化で厳密に計算した値と比較せよ．うまくいかない場合は、$T$や`optimized_state`をやり直すなどをしてみてください．

In [ ]:
from scipy.signal import find_peaks
# 複素数のオーバーラップを計算
overlap = overlap_re + 1j * overlap_im

# フーリエ変換を実行
overlap_fft = np.fft.fft(overlap)
freqs = np.fft.fftfreq(N_t, dt)

# フーリエスペクトルをシフトして中心にゼロ周波数を持ってくる
overlap_fft_shifted = np.fft.fftshift(overlap_fft)
freqs_shifted = np.fft.fftshift(freqs)

# スペクトルの絶対値を計算
spectrum = np.abs(overlap_fft_shifted)

# スペクトルをプロット
plt.xlim(0, 5)
plt.plot(freqs_shifted, spectrum)
plt.xlabel('Frequency')
plt.ylabel('Spectrum Magnitude')
plt.title('Fourier Transform of Overlap Function')
plt.show()

# ピークのインデックスを取得
peak_idx = np.argmax(spectrum)

# 対応する周波数を取得
peak_freq = freqs_shifted[peak_idx]

# スペクトル内のピークを検出
peaks, properties = find_peaks(spectrum, height=0)

# ピークに対応する周波数を取得
peak_freqs = freqs_shifted[peaks]
peak_magnitudes = spectrum[peaks]

# 正の周波数のみを考慮（エネルギーは正と仮定）
positive_freq_indices = np.where(peak_freqs > 0)
positive_peak_freqs = peak_freqs[positive_freq_indices]
positive_peak_magnitudes = peak_magnitudes[positive_freq_indices]



# 最小の正の周波数を持つピークを見つける
if len(positive_peak_freqs) > 0:
    min_energy_index = np.argmin(positive_peak_freqs)
    estimated_energy = positive_peak_freqs[min_energy_index]
    print("推定された最小エネルギー: ", estimated_energy*2*np.pi)
else:
    print("正の周波数のピークが見つかりませんでした。")

## 課題5：
上記のコード量子ビット数を5,10に変えて実行して結果を考察せよ．

## 課題6: 
授業の感想や要望をここに記載してください。